In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../../dataset/")
import torch
from torch.utils.data import DataLoader
import stats
from datasets import TrainDataset, TestDataset, ValDataset
import transforms as tran
from transformers import SegformerForSemanticSegmentation

In [3]:
path_to_results = '../models/segformer/results/'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
metrics = [stats.Precision(), stats.Recall(), stats.F2(), stats.DiceScore(), stats.Jac(), stats.Acc(), stats.Confidence(), stats.F1()]

In [4]:
train_for_cal = TrainDataset('../../../data/', transform=tran.ToTensor())
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])
count = len(train_for_cal) * 512 * 512
loader_for_cal = DataLoader(train_for_cal, batch_size=32, shuffle=False)

for inputs, mask in loader_for_cal:
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

t_ms = tran.Compose([
        tran.ToTensor(),
        tran.Normalize(total_mean, total_std)
    ])

In [5]:
def eval_model(pretrain, name, epoch, t):
    model = SegformerForSemanticSegmentation.from_pretrained(
        f"nvidia/{pretrain}", 
        num_labels=4,
    ).to(device)

    model.load_state_dict(state_dict=torch.load(f'{path_to_results}/{pretrain}_{name}/model_epoch_{epoch}.pth'))
    val = TestDataset('../data/', transform=t, with_background=True)
    val_loader = DataLoader(val, batch_size=1, shuffle=False)
    res = stats.calculate_statistics(model, device, val, val_loader, metrics)
    stats.print_statistics(res, metrics, f'{pretrain}_{name}')

In [6]:
eval_model('mit-b2', 'aug_flip_0.0001', "16" , t_ms)

model = SegformerForSemanticSegmentation.from_pretrained(
        f"nvidia/mit-b2", 
        num_labels=4,
    ).to(device)
model.load_state_dict(state_dict=torch.load(f'{path_to_results}/mit-b2_aug_flip_0.0001/model_epoch_16.pth'))


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b2 and are newly initialized: ['decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.batch_norm.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_fuse.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.linear_c.2.proj.bias', 'decode_head.batch_norm.running_mean', 'decode_head.classifier.weight', 'decode_head.linear_c.2.proj.weight', 'decode_head.batch_norm.running_var', 'decode_head.linear_c.1.proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([163, 8])
-----------------------------------
mit-b2_aug_flip_0.0001
8
	Precision: 0.7012% | ± 0.1926 (std) | ± 0.0371 (var)
	Recall: 0.7449% | ± 0.2073 (std) | ± 0.0430 (var)
	F2: 0.7034% | ± 0.2008 (std) | ± 0.0403 (var)
	DiceScore: 0.6821% | ± 0.1908 (std) | ± 0.0364 (var)
	Jac: 0.5452% | ± 0.1959 (std) | ± 0.0384 (var)
	Acc: 0.9631% | ± 0.0393 (std) | ± 0.0015 (var)
	Confidence: 0.9568% | ± 0.0230 (std) | ± 0.0005 (var)
	F1: 0.6821% | ± 0.1908 (std) | ± 0.0364 (var)
-----------------------------------


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b2 and are newly initialized: ['decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.batch_norm.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_fuse.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.linear_c.2.proj.bias', 'decode_head.batch_norm.running_mean', 'decode_head.classifier.weight', 'decode_head.linear_c.2.proj.weight', 'decode_head.batch_norm.running_var', 'decode_head.linear_c.1.proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>